In [2]:
import findspark
findspark.init()

In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [3]:
from pyspark import SparkContext
sc=SparkContext(master="local", appName="first app" )

## 맛보기

In [ ]:
# Action - collect()
rdd = sc.parallelize(range(1,11))
result = rdd.collect()
print(result) # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
id(result)    #139931150042440

In [ ]:
# Action - count()
rdd = sc.parallelize(range(1, 11))
result = rdd.count()
print(result)  # 10
id(result)     # 10914784

# Transformation
## Map과 관련된 연산들

In [ ]:
# transformation Action - map, collect()
rdd1 = sc.parallelize(range(1, 6))
rdd2= rdd1.map(lambda v : v+1)
print(rdd2.collect())
# [2, 3, 4, 5, 6]

In [ ]:
# map()
rdd1 = sc.parallelize(["apple,orange", "grape,apple,mango", "blueberry,tomato,orange"])
rdd2 = rdd1.map(lambda s: s.split(","))
print(rdd2.collect())
# [['apple', 'orange'], ['grape', 'apple', 'mango'], ['blueberry', 'tomato', 'orange']]

In [ ]:
# flatMap()
rdd1 = sc.parallelize(["apple,orange", "grape,apple,mango", "blueberry,tomato,orange"])
rdd2 = rdd1.flatMap(lambda s: s.split(","))
print(rdd2.collect())
# ['apple', 'orange', 'grape', 'apple', 'mango', 'blueberry', 'tomato', 'orange']

In [ ]:
# python코드 없이 만들어봄  # page 111
# flatMap return type TraversableOnce[U]   # None 리턴되는 것이 중요
rdd1 = sc.parallelize(["apple,orange", "grape,apple.mango", "blueberry,tomato,orange"])
def deflog(log):
    print(str(log)) # 실행되지 않음
    if "apple" == str(log):
        return list[log]
    else:
        return list(), print(type(log))
# deflog = [ if "apple" == str(log) ]
rdd2 = rdd1.flatMap(deflog)  # transformation에서는 error발생치 않음
print(rdd2)                  # PythonRDD[12] at RDD at PythonRDD.scala:53
print(rdd2.collect())       #  [[], None, [], None, [], None]

In [ ]:
# mapPartitions()
# map()과 flatMap() RDD의 각 요소를 하나씩 처리
# mapPartition() 파티션단위로 처리:파티션에 속한 모든 요소의 컬렉션에 대한 이터레이터(Iterator)를 입력으로 사용,리턴도 이터레이터
# 파티션 단위의 중간산출물을 만들거나 DB 연결과 같은 고비용의 자원을 파티션 단위로 공유해 사용할 수 있다는 장점

# increase
def increase(numbers):
    print("DB 연결 !!!")
    return(i + 1 for i in numbers)

#rdd1 = sc.parallelize(range(1,11))   # 옵션으로 partition 정할 수 있다
rdd1 = sc.parallelize(range(1,11), 3)   # 옵션으로 partition 정할 수 있다
rdd2 = rdd1.mapPartitions(increase)    # 함수자체를 매개변수로
print(rdd2.collect())
#[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [ ]:
# mapPartitionsWithIndex()
# 파티션에 속한 요소의 정보 + 해당파티션의 인덱스 함께 전달
#increaseWithIndex
def increaseWithIndex(idx, numbers):
    print("partitions !!!")
    for i in numbers:
        if(idx == 1):
            yield i + 1  # yield 일시정지 변수기억 next()

            
rdd1 = sc.parallelize(range(1,11), 3)
rdd2 = rdd1.mapPartitionsWithIndex(increaseWithIndex)  # 함수자체를 매개변수로
print(rdd2.collect())
# [5, 6, 7]  <== [ 4, 5, 6] 파티션이 index 1로 전달되었음을 추정할 수 있다

In [ ]:
# mapValue()  # RDD의 요소가 key value의 쌍으로 이루는 경우 페어RDD(PairRDD)
# 인자로 받은 value에 해당하는 요소에만 적용, 그 결과로 구성된 새로운 RDD 생성

rdd1 = sc.parallelize(["a", "b", "c"])

# PairRDD 생성
rdd2 = rdd1.map(lambda v : (v, 1))

rdd3 = rdd2.mapValues(lambda i: i + 1)
print(rdd3.collect())
# [('a', 2), ('b', 2), ('c', 2)]

In [ ]:
# flatMapValues()

rdd1 = sc.parallelize([(1, "a,b"), (2, "a,c"), (1, "d,e")])

# PairRDD 생성
rdd2 = rdd1.flatMapValues(lambda s: s.split(","))
print(rdd2.collect())
# [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'c'), (1, 'd'), (1, 'e')]

## 그룹과 관련된 연산들

In [ ]:
# zip()    # key value 생성 # 요소의 개수 같아야 한다
rdd1 = sc.parallelize(["a", "b", "c"])
rdd2 = sc.parallelize([1, 2, 3])

result = rdd1.zip(rdd2)
print(result) #org.apache.spark.api.java.JavaPairRDD@37a7333b
print(result.collect())
# [('a', 1), ('b', 2), ('c', 3)]

In [ ]:
# zipPartitions  파티션단위로 zip()연산 수행 # 파티션 개수 같아야 한다
# 파이썬에서는 사용할 수 없음


In [ ]:
# groupBy()   # value값을 새로운 key값으로 하는 (key, value그룹(시퀸스))으로 생성
# 인자로 전달하는 함수가 각 그룹의 키를 결정하는 역할 담당

rdd1 = sc.parallelize(range(1, 11))
rdd2 = rdd1.groupBy(lambda v: "even" if v%2==0 else "odd")
print(rdd2) # PythonRDD[38] at RDD at PythonRDD.scala:53
print(rdd2.collect())
#[('odd', <pyspark.resultiterable.ResultIterable object at 0x7f444277e860>), ('even', <pyspark.resultiterable.ResultIterable object at 0x7f444277e828>)]
for x in rdd2.collect():
    #print(x[0], x[1])
# odd <pyspark.resultiterable.ResultIterable object at 0x7f44432759e8>
# even <pyspark.resultiterable.ResultIterable object at 0x7f44604d7160>
    print(x[0], list(x[1]))
# odd [1, 3, 5, 7, 9]
# even [2, 4, 6, 8, 10]

In [ ]:
# groupByKey()  # RDD구성요소 key, value에서만 가능
# key기준으로 같은 키를 가진 요소들로 그룹을 만들고
rdd1 = sc.parallelize(["a", "b", "c", "b", "c"]).map(lambda v: (v, 1))
rdd2 = rdd1.groupByKey()
for x in rdd2.collect():
    print(x[0], list(x[1]))
# a [1]
# b [1, 1]
# c [1, 1]

In [ ]:
# cogroup()   ## RDD구성요소 key, value에서만 가능
# join과 비교
# 여러 RDD에서 같은 키를 갖는 값 요소를 찾아서 키와 그 키에 속하는 요소의 시퀀스(List, Vector 등의 상위클래스인 Iterable)
# 구성된 튜플, 그 튜플로 구성된 RDD 생성 
# [ Tuple(key, Tuple(rdd1요소들의 집합, rdd2요소들의 집합)), ... ]
rdd1 = sc.parallelize([("k1", "v1"), ("k2", "v2"), ("k1", "v3")])
rdd2 = sc.parallelize([("k1", "v4")])
result = rdd1.cogroup(rdd2)
print(result) # PythonRDD[120] at RDD at PythonRDD.scala:53
print(result.collect())
# [('k2', (<pyspark.resultiterable.ResultIterable object at 0x7f4440be7e48>, <pyspark.resultiterable.ResultIterable object at 0x7f4440be7c50>)), ('k1', (<pyspark.resultiterable.ResultIterable object at 0x7f4440be7dd8>, <pyspark.resultiterable.ResultIterable object at 0x7f4440be4f28>))]
for x in result.collect():
    print(x[0], list(x[1][0]), list(x[1][1]))
# k2 ['v2'] []
# k1 ['v1', 'v3'] ['v4']

### 집합과 관련된 연산들

In [ ]:
# distinct()  # RDD의 원소에서 중복을 제외한 요소로만 구성된 새로운 RDD 생성
rdd = sc.parallelize([1,2,3,1,2,3,1,2,3])
result = rdd.distinct()
print(result.collect())
#[1, 2, 3]

In [ ]:
# cartesian()  # RDD구성요소 key, value에서만 가능
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize(["a", "b", "c"])
result = rdd1.cartesian(rdd2)
print(result.collect())
#[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c'), (3, 'a'), (3, 'b'), (3, 'c')]

In [ ]:
# subtract()  # 두 개의 RDD가 있을 때 rdd1- rdd2
rdd1 = sc.parallelize(["a", "b", "c", "d", "e"])
rdd2 = sc.parallelize(["d", "e"])
result = rdd1.subtract(rdd2)
print(result.collect())
#['b', 'c', 'a']

In [ ]:
# union()
rdd1 = sc.parallelize(["a", "b", "c"])
rdd2 = sc.parallelize(["d", "e", "f"])
result = rdd1.union(rdd2)
print(result.collect())
#['a', 'b', 'c', 'd', 'e', 'f']

In [ ]:
# intersection()  #  중복제외됨
rdd1 = sc.parallelize (["a", "a", "b", "c" ])
rdd2 = sc.parallelize (["a", "a", "c", "c"])
result = rdd1.intersection(rdd2)
print(result.collect())
# ['c', 'a']

In [ ]:
# join()  # RDD구성요소 key, value에서만 가능
# cogroup와 비교  # Action값을 바로 볼 수 있다
# [ Tuple(key, Tuple(rdd1요소들의 집합, rdd2요소들의 집합)), ... ]
rdd1 = sc.parallelize (["a", "b", "c", "d", "e"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b", "c"]).map(lambda v: (v, 2))
result = rdd1.join(rdd2)
print(result.collect())
#[('b', (1, 2)), ('c', (1, 2))]

In [ ]:
# leftOuterJoin(), rightOuterJoin() # RDD구성요소 key, value에서만 가능
# key기준으로 외부조인을 수행
rdd1 = sc.parallelize (["a", "b", "c", "d", "e"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b", "c"]).map(lambda v: (v, 2))
result1 = rdd1.leftOuterJoin(rdd2)
result2 = rdd1.rightOuterJoin(rdd2)
print(f'leftOuterJoin: {result1.collect()}')
print(f'rightOuterJoin: {result2.collect()}')
# leftOuterJoin: [('b', (1, 2)), ('c', (1, 2)), ('d', (1, None)), ('a', (1, None)), ('e', (1, None))]
# rightOuterJoin: [('b', (1, 2)), ('c', (1, 2))]

In [ ]:
# subtractByKey()  # RDD구성요소 key, value에서만 가능
rdd1 = sc.parallelize (["a", "b"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b"]).map(lambda v: (v, 1))
result = rdd1.subtractByKey(rdd2)
print(result.collect())
# [('a', 1)]

## 집계와 관련된 연산들

In [ ]:
# reduceByKey()  # RDD구성요소 key, value에서만 가능
# 2개의 값을 하나로 합치는 함수를 인자로 전달받는데, 이 함수는 요소들이 결합, 교환법칙 성립되어야 한다
rdd = sc.parallelize (["a", "b", "b"]).map(lambda v: (v,1))
result = rdd.reduceByKey(lambda v1, v2: v1 + v2)
print(result.collect())
# [('a', 1), ('b', 2)]

In [ ]:
# foldByKey()   # RDD구성요소 key, value에서만 가능
# reduceByKey와는 달리 병합연산의 초기값을 메서드의 인자로 전달해서 병합시 사용
# ex) 더하는 함수 0, 두 문자열 연결 공백문자""를 초기값으로 사용가능
# 하지만 이 때 초기값이 반복해도 연산결과에 영향을 주지 않는 값이여야 함
# 함수는 교환법칙은 만족안해도 되고 결합법칙은 만족해야 함

rdd = sc.parallelize (["a", "b", "b"]).map(lambda v: (v,1))
result = rdd.foldByKey(0, lambda v1,v2:v1+v2)
print(result.collect())
# [('a', 1), ('b', 2)]

In [ ]:
# error  # page 137
# combineByKey()    # RDD구성요소 key, value에서만 가능
class Record:
        
    def __init__(self, amount, number=1):
        self.amount = amount
        self.number = number
        
    def addAmt(self, amount):
        return Record(self.amount + amount, self.number + 1)
    
    def __add__(self, other):
        amount = self.amount + other.amount
        number = self.number + other.number 
        return Record(amount, number)
        
    def __str__(self):
        return "avg:" + str(self.amount / self.number)

    def __repr__(self):
        return 'Record(%r, %r)' % (self.amount, self.number)
        
# combineBy
def createCombiner(v):
    return Record(v)

# combineBy
def mergeValue(c, v):
    return c.addAmt(v)

# combineBy
def mergeCombiners(c1, c2):
    return c1 + c2

rdd = sc.parallelize([("Math", 100), ("Eng", 80), ("Math", 50), ("Eng", 70), ("Eng", 90)])
result = rdd.combineByKey(lambda v: createCombiner(v), lambda c, v: mergeValue(c, v), lambda c1, c2: mergeCombiners(c1, c2))
print('Math', result.collectAsMap()['Math'], 'Eng', result.collectAsMap()['Eng'])
#print(str(result.collectAsMap()))

In [ ]:
# aggregateByKey()   # RDD구성요소 key, value에서만 가능
# combineByKey()와 동일하나 초기값 생성하는 부분만 다름
# result = rdd.aggregateByKey(zero)(mergeValue, mergecombiners)

## pipe 및 파티션과 관련된 연산

In [ ]:
# pipe()
rdd = sc.parallelize( ["1,2,3", "4,5,6", "7,8,9"])
result = rdd.pipe("cut -f 1,3 -d ,") # -field  1,3출력 구분자 -d=delimeter default tab, 여기서는 ,
print(result.collect())
# ['1,3', '4,6', '7,9']

In [ ]:
# 최초 설정한 partition 수 조정필요시
# coalesce()     : 줄이기만 가능         셔플 수행하라는 옵션 지정하지 않는 : 셔플을 사용하지 않기 때문
# repartition() : 늘리거나 줄일 수 있음  셔플 기반으로 동작
rdd1= sc.parallelize(list(range(1, 11)),10)
rdd2 = rdd1.coalesce(5)
rdd3 = rdd2.repartition(10)
print(f'partition size: {rdd1.getNumPartitions()}')
print(f'partition size: {rdd2.getNumPartitions()}')
print(f'partition size: {rdd3.getNumPartitions()}')

In [ ]:
# page 142
# repartitionAndSortWithinPartitions()    # RDD구성요소 key, value에서만 가능
# RDD를 구성하는 모든 데이터를 특정 기준에 따라 여러 개의 파티션으로 분리, 각 파티션단위로 정렬수행한 뒤, 
# 그 결과로 새로운  RDD을  생성
# 메서드 실행시 각 데이터가 어떤 파티션에 속할지 결정하기 위한 파티셔너(Partitioner)를 설정
# 파티셔너는 각 데이터의 키 값을 이용해 데이터가 속할 파티션 결정, 키 값을 이용한 정렬도 함께 수행
# 즉 파티션 재할당을 위해 셔플을 수행하는 단계에서 정렬도 함께 다루게 되어 파티션과 정렬 따로하는 것보다 높은 성능

import random
data = [random.randrange(1,100) for i in range(0, 10)]
#print(data) # [23, 99, 61, 18, 84, 58, 75, 13, 55, 47]

rdd1 = sc.parallelize(data).map(lambda v: (v, "-"))
#print(rdd1.collect())  # [(73, '-'), (79, '-'), (47, '-'), (98, '-'), (33, '-'), (51, '-'), (43, '-'), (68, '-'), (95, '-'), (69, '-')]

rdd2 = rdd1.repartitionAndSortWithinPartitions(3, lambda x: x) # HashPartitioner 파티셔너 3
# print(rdd2) # PythonRDD[187] at RDD at PythonRDD.scala:53
# rdd2.count # <bound method RDD.count of PythonRDD[315] at RDD at PythonRDD.scala:53>

# 아래코드 에러는 없으나 결과 출력물이 안나옴  # cli에서는 나옴 (foreach관련) # foreach(함수)
# rdd2.foreachPartition(lambda values: print(list(values))) # RDD의 파티션단위로 특정함수를 실행
# [Stage 0:> (0 + 1) / 1][(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# [(49, '-'), (52, '-'), (58, '-')]
# [(35, '-'), (53, '-'), (89, '-')]

rdd2.foreachPartition(  lambda values: print( list( map(lambda v: v, values)) ) ) 
# [(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# [(49, '-'), (52, '-'), (58, '-')]
# [(35, '-'), (53, '-'), (89, '-')]


#아래코드 에러는 없으나 결과 출력물이 안나옴   # cli 에서는 나옴 (foreach)
# def test(it):
#     print("="*5)
#     print ( list( map(lambda x: x, it) ) )
# rdd2.foreachPartition(test) 
# =====
# [(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# =====
# [(49, '-'), (52, '-'), (58, '-')]
# =====
# [(35, '-'), (53, '-'), (89, '-')]


# 아래코드는 Py4JJavaErro # cli에서도 error # foreach 2번
# rdd2.foreachPartition( lambda it: it.foreach(lambda v: print(v))  ) # RDD의 파티션단위로 특정함수를 실행

# 아래코드는 Py4JJavaErro # error
# def test(it):
#     print("="*5)
#     it.foreach( lambda x: print(x) )             
# rdd2.foreachPartition(test)  



In [ ]:
# partitionBy()   # RDD구성요소 key, value에서만 가능 
# 파티셔너(Partitioner) 인자로 전달
# HashPartitioner   : 각 요소의 키 값으로부터 해시값을 취해 이 값을 기준으로 파티션을 결정하는 방법
# Rangepartitioner : 순서가 있는 요소들(Sortable)로 구성된 RDD에 사용가능,
#                    각 요소를 목표 파티션 크기에 맞게 일정크기의 구간으로 나누는 방식

rdd1 = sc.parallelize( [ ("apple", 1), ("mouse", 1), ("monitor", 1)],  5 )
rdd2 = rdd1.partitionBy(3)
print(f'rdd1: {rdd1.getNumPartitions()} ')  # rdd1: 5 
print(f'rdd2: {rdd2.getNumPartitions()} ')  # rdd2: 3 


In [ ]:
# list ( map (함수, iter) ) 연습

# li = [1, 2, 3]
# result = map(lambda i: i * i, li)
# print(next(result))  # 1
# print(next(result))  # 1
# print( list(map(lambda i: i * i, li)) )  #[1, 4, 9]


## 필터와 정렬 연산

In [ ]:
# filter()   # python filter # list( filter(함수,iter) )
rdd1 = sc.parallelize(range(1,6))
rdd2 = rdd1.filter(lambda i : i > 2)
print(rdd2.collect())
# [3, 4, 5]

In [ ]:
# sortByKey()   # RDD구성요소 key, value에서만 가능
# 키 값을 기준으로 요소를 정렬하는 연산  # 소팅 완료 후 파티션 내부의 요소는 소팅 순서상 인접한요소로 재구성
rdd = sc.parallelize( [("q",1), ("z", 1), ("a", 1)])
result = rdd.sortByKey()
print(result.collect())
# [('a', 1), ('q', 1), ('z', 1)]

In [ ]:
# keys()  키에 해당하는 요소로 구성된 RDD 생성
# values() 값에 해당하는 요소로 구성된 RDD 생성
rdd = sc.parallelize([("k1", "v1"), ("k2", "v2"), (" k3", "v3")])
result1 = rdd.keys()
result2 = rdd.values()
print(result1.collect()) # ['k1', 'k2', ' k3']
print(result2.collect()) # ['v1', 'v2', 'v3']

In [ ]:
# sample()
# 스칼라 API    sample(withReplacement: Boolean, fraction: Double, seed: Long=Utils.nextLong): RDD[t]
# withReplacement : 복원추출 수행여부 True, False 
# fraction 복원일경우 : 샘플내에서 각 요소가 나타나는 횟수에 대한 기대값, 즉 각요소의 평균발생 횟수, 0이상
#          비복원일경우:          각 요소가 샘플에 포함될 확률, 0 ~ 1 사이 값으로 지정가능

# 샘플의 크기를 정해놓고 추출을 실행하지 않음  fraction 0.5 전체크기 50% 샘플추출아님
# 크기를 정해놓은 메소드 takeSample()
# seed : 일반적인 무작위 값 추출시 사용하는 것과 유사한 개념으로 반복시행시 결과가 바뀌지 않도록 제어 목적

rdd = sc.parallelize(range(1, 101))
result1 = rdd.sample(False, 0.5, 666) 
result2 = rdd.sample(True, 1.5, 777)
print(f'result1.count(): {result1.count()}  result1.take(5): {result1.take(5)}') 
# result1.count(): 54  result1.take(5): [2, 4, 5, 6, 7]
print(f'result2.count(): {result2.count()}  result2.take(5): {result2.take(5)}') 
# result2.count(): 153  result2.take(5): [1, 4, 4, 4, 4]

# ACTION 
## 출력과 관련된 연산

In [ ]:
# Lazy evaluation  # 함수형 프로그래밍 자주 활용  
# 액션연산시 트랜스포메이션 연산도 반복수행됨. # 반복 수행 성능을 개선하기 위한 방안(캐시)

In [ ]:
# first()
# RDD요소가운데 첫번째 요소 하나를 return
rdd = sc.parallelize( [5, 4, 1])
result = rdd.first()
print(result) # 5   # first() Action이므로 value return

In [ ]:
# take()
# RDD의 첫번째 요소부터 n개를 추출해서 return
rdd = sc.parallelize (range(1,21), 5)
print ( rdd.take(5) ) # [1, 2, 3, 4, 5]

In [ ]:
# takeSample()
# RDD 요소 가운데 지정된 크기의 샘플을 추출하는 메서드
# sample()과 차이: 1. 샘플의 크기 지정가능   2. 결과 타입이 RDD가 아닌 배열이나 리스트같은 컬렉션 타입

rdd = sc.parallelize(range(1, 101))
result = rdd.takeSample(False, 20, 666)
print(result)
# [76, 26, 83, 72, 93, 42, 92, 50, 18, 46, 21, 70, 40, 65, 67, 87, 7, 62, 54, 37]

In [ ]:
# collect(), count()
# countByValue()  RDD에 속하는 각 값들이 나타나는 횟수를 구해서 맵(dict) 형태로 돌려주는 메서드
rdd = sc.parallelize( ["a","a","b","c","c",])
result = rdd.countByValue()
print(result) # defaultdict(<class 'int'>, {'a': 2, 'b': 1, 'c': 2})
for k, v in result.items():
    print(k,v)
# a 2
# b 1
# c 2

In [ ]:
# reduce()
# RDD에 포함된 임의의 값 2개를 하나로 합치는 함수를 이용해, RDD에 모든 요소를 하나의 값으로 병합하고 그 결과 반환
# 각 서버에 흩어져 있는 파티션 단위로 처리 ==> RDD 모든 요소 교환,결합법칙 성립에만 사용 가능 (파티션별로 계산하므로, 나누기 안됨)
rdd = sc.parallelize(range(1, 11), 3)
result = rdd.reduce(lambda v1, v2: v1+v2)  # 1+2+3  4+5+6   7+8+9
print(result) # 55

In [ ]:
#fold()   # reduce()와 동일하고
# 병합 연산의 초기값 지정해 줄 수 있다 : 덧셈 0, 곱셈 1
rdd = sc.parallelize(range(1, 11), 3)
result = rdd.fold(0, lambda v1, v2: v1+v2)
print(result) # 55

# 파이썬 코드없어 작성 error py4jJavaError  # page 155 
class Prod():
    #self.cnt = 1
    def __init__(self, price):
        self.price = price
        self.cnt = 1
def reduce_(p1, p2):
        p1.price += p2.price
        p1.cnt += 1   
def fold_(p1, p2) :
    p1.price += p2.price
    p1.cnt += 1    
def reduceVsFold(sc):
    rdd = sc.parallelize([Prod(300), Prod(200), Prod(100)], 10) # 파티션이 10ea, 객체 3ea
    # print(rdd) #ParallelCollectionRDD[35] at parallelize at PythonRDD.scala:195
    
    r1 = rdd.reduce(reduce_)
    print(f'Reduce: {r1.price}, {r1.cnt}')
    
    r2 = rdd.fold(Prod(0), fold_)  # 초기값에 의해 요소가 없더라도 초기값으로 인핸 연산수행
    print(f'fold: {r2.price}, {r2.cnt}')    
    
reduceVsFold(sc)

In [ ]:
# aggregate()   # error Py4JJavaError
# reduce() fold() 모두 입력과 출력 타입이 동일해야 한다는 제약, aggregate() 입출력 타입이 다를 경우 사용가능
# 인자 3ea : 초기값, seqOp 각 파티션단위 부분합 병합함수, combOp 최종적 하나로 합치는 병합함수
# seqOp, combOp 이전 단계의 병합 결과로 생성된 객체를 재사용하기에 매번 새로운 객체가 생성되는 부담을 덜 수 있다

# error Py4JJavaError
sc.stop()
class Record:

    def __init__(self, amount, number=1):
        self.amount = amount
        self.number = number
        
    def addAmt(self, amount):
        return Record(self.amount + amount, self.number + 1)
    
    def __add__(self, other):
        amount = self.amount + other.amount
        number = self.number + other.number 
        return Record(amount, number)
        
    def __str__(self):
        return "avg:" + str(self.amount / self.number)

    def __repr__(self):
        return 'Record(%r, %r)' % (self.amount, self.number)
    
# Aggregate
def seqOp(r, v):
    return r.addAmt(v)

# Aggregate
def combOp(r1, r2):
    return r1 + r2

class RDDOpSample():
    def doAggregate(self, sc):
        rdd = sc.parallelize([100, 80, 75, 90, 95])
        result = rdd.aggregate(Record(0, 0), seqOp, combOp)
        print(result)
        
if __name__ == "__main__":
    import findspark
    findspark.init()
    from pyspark import SparkContext, SparkConf

#     conf = SparkConf()
    sc = SparkContext(master="local[*]", appName="RDDOpSample")

    obj = RDDOpSample()
    obj.doAggregate(sc)

In [ ]:
# sum() : RDD 모든 요소가 double, Long 등 숫자타입일 경우에만 사용, 전체 합
# 특화함수: RDD 키 값 에서 사용하는 reduceByKey(), combineByKey()
rdd = sc.parallelize(range(1,11))
result = rdd.sum()
print(result) # 55

In [ ]:
# foreach(), foreachPartition()
# 모든 요소에 하나씩 특정함수(인자)를 적용하는 메서드 : 실행결과 돌려주지 않음 
# 각 word node에서 동작하므로 print()사용하면 각 work node 콘솔에 보여준다
# mapPartition(), mapPartitionsWithIndex() : 실행 결과 돌려줌

# cli에서 실행 (python3)
def sideEffect(values):
    print("Partition side Effect")
    for v in values:
        print(f'Value Side Effect:{v}')
rdd = sc.parallelize(range(1,11), 3)
result = rdd.foreach(lambda v: print(f'Value Side Effect: {v}'))
# [Stage 0:> (0 + 1) / 3]Value Side Effect: 1
# Value Side Effect: 2
# Value Side Effect: 3
# Value Side Effect: 4
# Value Side Effect: 5
# Value Side Effect: 6
# Value Side Effect: 7
# Value Side Effect: 8
# Value Side Effect: 9
# Value Side Effect: 10

result = rdd.foreachPartition(sideEffect)
# Partition side Effect
# Value Side Effect:1
# Value Side Effect:2
# Value Side Effect:3
# Partition side Effect
# Value Side Effect:4
# Value Side Effect:5
# Value Side Effect:6
# Partition side Effect
# Value Side Effect:7
# Value Side Effect:8
# Value Side Effect:9
# Value Side Effect:10


In [ ]:
# toDebugString()
# RDD 파티션 개수나 의존성 정보 등 세부정보를 알고 싶을 때
rdd = sc.parallelize(range(1, 101), 10).map(lambda x: x*2).persist().map(lambda x:x+1).coalesce(2)
print(rdd.toDebugString())
# b'(2) CoalescedRDD[10] at coalesce at NativeMethodAccessorImpl.java:0 []\n    # 최종 파티션 2
# |  PythonRDD[9] at RDD at PythonRDD.scala:53 []\n
# |  PythonRDD[8] at RDD at PythonRDD.scala:53 []\n
# |  ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

In [ ]:
# cache()    RDD의 데이터를 메모리에 저장하라는 의미로, 메모리 부족시 그 부족한 용량만큼 수행하지 않음
# persist()  StorageLevel이라는 옵션을 이용해 저장위치와 저장방식(직렬화 여부) 등을 상세히 지정할 수 있음
# unpersist() 이미 저장중인 데이터가 더 이상 필요없을 때 캐시설정 취소
rdd = sc.parallelize(range(1, 101), 10)
rdd.cache()
from pyspark.storagelevel import StorageLevel
rdd.persist(StorageLevel.MEMORY_ONLY)
# PythonRDD[14] at RDD at PythonRDD.scala:53

# 캐시정첵
# 스파크는 메모리를 일정한 비율로 데이터 처리위한 공간, 캐시 저장 공간으로 활용 => 모두 부족시 처리위한 공간에 우선 할당
# 데이터의 특성과 용량, 사용 가능한 클러스터 자원을 신중히 고려하여 정책 수립해야 함

In [ ]:
# partitions()  : RDD의 파티션 정보가 담긴 배열을 돌려줌  # 파이썬은 사용할 수 없음
# getNumPartitions() : 단순한 크기정보만 알아 볼 목적에   # 파이썬 가능

rdd = sc.parallelize(range(1, 101), 10)
#print(rdd.partitions())
print(rdd.getNumPartitions()) #10

## RDD 데이터 불러오기와 저장하기

In [ ]:
# 파일포맷 : 텍스트 파일, json, 하둡의 시퀸스파일, csv
# 파일시스템 : local, HDFS, AWS의 S3, 오픈스택의 swift
# SQL, NoSQL : MySQL,        HBase, 카산드라, Hive

In [9]:
!pwd

/home/pyspark/work


In [25]:
# 텍스트 파일  # textFile()  # 파티션 지정 가능
# file://path
# hdfs://master:port/path/...
# s3n://bucket/path
# 모든 서버에 지정된 경로를 통해 지정한 파일에 접근할 수 있어야 한다

rdd = sc.parallelize(range(1, 1001), 3)
codec = "org.apache.hadoop.io.compress.GzipCodec" # 압축(gzip)  # gzip, Snappy, bzip2, LZO 가능

# save   # 동일한 폴더있으면 error # 다시 실행시 삭제후 실행  # dataframe, dataset에서는 append or overwrite기능 제공
rdd.saveAsTextFile("./sub1")         # part-00000.crc ,,3개 파일로 나뉘어 저장됨(하둡과 같은 방법)
rdd.saveAsTextFile("./sub2", codec) # 압축(gzip)

# path
# path=%pwd
# print(path) # /home/pyspark/work
# py용 path
import os
path=os.getcwd()
print(path) # home/pyspark/work

#load
rdd2 = sc.textFile(f'file://{path}/sub1')
print(rdd2.take(5))
# ['334', '335', '336', '337', '338']

/home/pyspark/work
/home/pyspark/work
['334', '335', '336', '337', '338']


In [26]:
# 오브젝트 파일 objectFile  # 파이썬에서는 saveAsPickleFile(), pickleFile() 사용
# SparkContext()의 objectFile(), RDD의 saveAsObjectFile() : object 직렬화방법을 이용해 RDD 구성하는 요소를 파일에 읽고 쓰기 기능 수행
# 자주 사용하지 않는 이유 : 속도가 느리고 변경에 취약하다
import os
path=os.getcwd()
print(path) # home/pyspark/work

rdd = sc.parallelize(range(1, 1000), 3)
rdd.saveAsPickleFile(f'file://{path}/object') # 저징
rdd2 = sc.pickleFile(f'file://{path}/object') # 저장된 RDD의 타입으로 , 모두 string으로 읽어지는 것이 아님
print(rdd2.take(5))
# [334, 335, 336, 337, 338]

/home/pyspark/work
[334, 335, 336, 337, 338]


In [30]:
# Sequence file
# 키와 값으로 구성된 데이터를 저장하는 이진Binary 파일 포맷, 하둡의 대표적 파일 포맷
# 오브젝트 파일(자바 표준 직렬화)도 이진 파일 포맷이지만, sequence는 하둡 자체 프레임워크
# 오브젝트 파일 : 자바의 Serializable 인터페이스 구현, # sequence 파일 RDD 데이터는 하둡의 Writable 인터페이스 구현

import os
path=os.getcwd()
path=path+'/'+'python'  # 새로운 폴더를 만들어 주어야 에러나지 않음 # 다시 실행시 삭제후 실행해야 함
# print(path) # home/pyspark/work

outputFormatClass = "org.apache.hadoop.mapreduce.lib.output.SequenceFileOutputFormat"
inputFormatClass = "org.apache.hadoop.mapreduce.lib.input.SequenceFileInputFormat"
keyClass = "org.apache.hadoop.io.Text"
valueClass = "org.apache.hadoop.io.IntWritable"
conf = "org.apache.hadoop.conf.Configuration"

rdd1 = sc.parallelize(["a", "b", "c", "b", "c"])
rdd2 = rdd1.map(lambda x: (x, 1))

# save
rdd2.saveAsNewAPIHadoopFile(path, outputFormatClass, keyClass, valueClass)

#load
rdd3 = sc.newAPIHadoopFile(path, inputFormatClass, keyClass, valueClass)
for k, v in rdd3.collect():
    print(k, v)
# a 1
# b 1
# c 1
# b 1
# c 1

a 1
b 1
c 1
b 1
c 1


In [31]:
# 클러스터 환경에서의 공유 변수
# 다수의 프로세스가 공유할 수 있는 읽기 자원과 쓰기 자원을 설정할 수 있도록 지원
# 하둡 : 분산캐시(distributedCache)와 카운터(counter)   ===================> 단순
# 스파크 : 브로드캐스트(Broadcast Variables)와 어큐물레이터(Accumulators) => 범용
# Broadcast Variables : 읽기 전용 자원 (ex 온라인 쇼핑몰 : set컬렉션 타딥의 데이터 공유변수 설정, 각 서버에서 로그처리하면서 사용자확인)
# Accumulators Variables : 쓰기 자원 (ex 온라인 쇼핑몰: 기록할 로그형식 정함)

bu = sc.broadcast(set(["u1", "u2"]))
rdd = sc.parallelize(["u1", "u3", "u3", "u4", "u5", 'u6'], 3)
result = rdd.filter(lambda v: v in bu.value)
print(result.collect())
# ['u1']

# 스파크는 액션 연산을 수행시 동일한 스테이지(stage)내에서 실행되는 태스크(task)간에는 동작에 필요한 변수를 자동으로
# 브로드캐스트 변수를 지정하지 않아도 된다. 즉 여러 스테이지에 활용되는 경우가 아니면 굳이 명시적으로 브로드캐스트 변수 지정할 필요없음
# 액션 연산이 수행될 때 reduceByKey()와 같이 셔플이 발생돼야 하는 경우마다 새로운 스테이지가 시작됨.
# 즉 어떤 연산을 수행하는 과정에서 셔플을 발생시키지 않고 로컬 서버내에서 처리될 수 있는 태스크는 하나의 스테이지에 속함

['u1']


In [ ]:
# 어큐뮬레이터 Accumulators
# ex) 모든 서버의 오류정보를 모아서 볼수 있게하는 것 => 각 서버에서 발생하는 특정 이벤트의 수를 세거나 관찰하고 싶은 정보
# 


In [42]:
sc.stop()

In [43]:
from pyspark import SparkContext, SparkConf, AccumulatorParam
from builtins import isinstance

class Record:

    def __init__(self, amount, number=1):
        self.amount = amount
        self.number = number
        
    def addAmt(self, amount):
        return Record(self.amount + amount, self.number + 1)
    
    def __add__(self, other):
        amount = self.amount + other.amount
        number = self.number + other.number 
        return Record(amount, number)
        
    def __str__(self):
        return "avg:" + str(self.amount / self.number)

    def __repr__(self):
        return 'Record(%r, %r)' % (self.amount, self.number)
    
class AccumulatorSample():

    # ex 2-142
    def runBuitInAcc(self, sc):
        acc1 = sc.accumulator(0)
        data = ["U1:Addr1", "U2:Addr2", "U3", "U4:Addr4", "U5;Addr5", "U6:Addr6", "U7::Addr7"]
        rdd = sc.parallelize(data)
        rdd.foreach(lambda v: accumulate(v, acc1))
        print(acc1.value)

    # ex 2-145
    def runCustomAcc(self, sc):
        acc = sc.accumulator(Record(0), RecordAccumulatorParam())
        data = ["U1:Addr1", "U2:Addr2", "U3", "U4:Addr4", "U5;Addr5", "U6:Addr6", "U7::Addr7"]
        rdd = sc.parallelize(data)
        rdd.foreach(lambda v: accumulate(v, acc))
        print(acc.value.amount)


def accumulate(v, acc):
    if (len(v.split(":")) != 2):
        acc.add(1)


class RecordAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return Record(0)

    def addInPlace(self, v1, v2):
        if (isinstance(v2, Record)):
            return v1 + v2
        else:
            return v1.addAmt(v2)


if __name__ == "__main__":
    conf = SparkConf()
    sc = SparkContext(master="local[*]", appName="AccumulatorSample", conf=conf)

    obj = AccumulatorSample()
    obj.runBuitInAcc(sc)  # 3
    #obj.runCustomAcc(sc)  # error
    sc.stop()


3
